# Imports And Libraries

In [6]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *

# import pandas as pd
from boto import get_square_secret

# import os
import pickle

# Generic Functions

In [2]:
def get_value(key1,key2):
    try:
        return key1[key2] 
    except:
        return None

# Get Existing Subs

In [3]:
SAMPLE_SPREADSHEET_ID_input = '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8'
SAMPLE_RANGE_NAME = 'Square Subs!A:G'

exist = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)


# Get Data of most Recent Sub

In [4]:
dates = exist[exist['Date'].str.contains('T')]

most_recent_transaction_date = pd.to_datetime(dates['Date'], format='mixed').max().isoformat()

most_recent_transaction_date

'2023-09-10T20:15:23+00:00'

# Get New Subs

In [7]:
client = Client(
    access_token=get_square_secret()['SQUARE_ACCESS_TOKEN'],
    environment='production')


In [8]:
orders_api = client.orders
customers_api = client.customers

In [9]:
most_recent_transaction_date

'2023-09-10T20:15:23+00:00'

In [14]:
body = {
    'location_ids': [
        'SBBJSGR24YHYB'
    ],
    'query': {
        'filter': {
            
            'date_time_filter': {
                'updated_at': {
                    'start_at': f'2023-09-06T20:15:23+00:00',
                }
            },
      # "state_filter": {
        # "states": [
        #   "COMPLETED"
        # ]
      # }
       },
    "sort": {
      "sort_field": "UPDATED_AT",
      "sort_order": "DESC"
    }
        
    },
   }

In [15]:
s_orders = orders_api.search_orders(body).body['orders']

len(s_orders)

357

In [47]:
ls=[x['line_items'] for x in s_orders]

a=[x for x in ls if 'atch' not in x[0]['name']]
b=[x for x in a if 'Reg' not in x[0]['name']]
b=[x for x in b if '23' not in x[0]['name']]
b
# a
# ls

[[{'uid': 'hKFn42MMdPLLKnzZwEwpG',
   'quantity': '1',
   'name': 'Training',
   'base_price_money': {'amount': 500, 'currency': 'GBP'},
   'gross_sales_money': {'amount': 500, 'currency': 'GBP'},
   'total_tax_money': {'amount': 0, 'currency': 'GBP'},
   'total_discount_money': {'amount': 0, 'currency': 'GBP'},
   'total_money': {'amount': 500, 'currency': 'GBP'},
   'variation_total_price_money': {'amount': 500, 'currency': 'GBP'},
   'item_type': 'ITEM',
   'total_service_charge_money': {'amount': 0, 'currency': 'GBP'}}],
 [{'uid': 'hm4hYrL1idfdDH4j1Hgyc',
   'quantity': '1',
   'name': 'Training',
   'base_price_money': {'amount': 500, 'currency': 'GBP'},
   'gross_sales_money': {'amount': 500, 'currency': 'GBP'},
   'total_tax_money': {'amount': 0, 'currency': 'GBP'},
   'total_discount_money': {'amount': 0, 'currency': 'GBP'},
   'total_money': {'amount': 500, 'currency': 'GBP'},
   'variation_total_price_money': {'amount': 500, 'currency': 'GBP'},
   'item_type': 'ITEM',
   'tot

#  Get All Invoices

In [16]:
invoices_api = client.invoices


result = invoices_api.list_invoices(
    'SBBJSGR24YHYB'
)

invoices = pd.DataFrame(result.body['invoices'])

i=1
while result.cursor:
    print(i)
    i+=1
    result = invoices_api.list_invoices(
        'SBBJSGR24YHYB',
        cursor = result.cursor
    )
invoices = pd.concat([invoices, pd.DataFrame(result.body['invoices'])])
invoices


1
2


,id,version,location_id,order_id,payment_requests,primary_recipient,invoice_number,title,description,scheduled_at,status,timezone,created_at,updated_at,accepted_payment_methods,delivery_method,sale_or_service_date,store_payment_method_enabled,public_url,next_payment_amount_money
0,inv:0-ChDKYvPVrRG6pCpBzngISHdQEIMP,1,SBBJSGR24YHYB,hrvB92DS41uvA7C6znatRYqfebeZY,[{'uid': 'a93dbe83-18e2-4d88-9877-59a5fa6156df...,"{'customer_id': '17C7YC49EWVW1CYAYMVVF0W3DM', ...",2023-09-11_Vets_hrvB92DS41uvA7C6znatRYqfebeZY_1,Match Subs 09 Sep vs Motspur Park Vets,Match Subs 09 Sep vs Motspur Park Vets,2023-09-12T10:00:00Z,SCHEDULED,Europe/London,2023-09-11T09:35:07Z,2023-09-11T09:35:32Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-09-11,False,NaN,NaN
1,inv:0-ChDxYEB6eZy2R00BCKPOhYATEIMP,1,SBBJSGR24YHYB,Zz3GjoR9FjgIFjbFDOMr7cshM1HZY,[{'uid': 'de98a2f8-13e9-4a41-a188-1e66ee5edf10...,"{'customer_id': '9JF8RAVFSN5TAY187PRHRA6NN0', ...",2023-09-11_Vets_Zz3GjoR9FjgIFjbFDOMr7cshM1HZY_1,Match Subs 09 Sep vs Motspur Park Vets,Match Subs 09 Sep vs Motspur Park Vets,2023-09-12T10:00:00Z,SCHEDULED,Europe/London,2023-09-11T09:35:06Z,2023-09-11T09:35:32Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-09-11,False,NaN,NaN
2,inv:0-ChAUYaWPDIpFEK28jW7EDSHwEIMP,1,SBBJSGR24YHYB,BXBNO2Laj2uA8B8sRbKDeRbBey6YY,[{'uid': 'd8c2d065-95cb-4d77-9ff3-2d541d9f2ba7...,"{'customer_id': 'MJCB0H07T5VYS60WTN6KQZ4QJR', ...",2023-09-11_Vets_BXBNO2Laj2uA8B8sRbKDeRbBey6YY_1,Match Subs 09 Sep vs Motspur Park Vets,Match Subs 09 Sep vs Motspur Park Vets,2023-09-12T10:00:00Z,SCHEDULED,Europe/London,2023-09-11T09:35:05Z,2023-09-11T09:35:31Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-09-11,False,NaN,NaN
3,inv:0-ChBvpmoww5AxIGwaF8oa3887EIMP,1,SBBJSGR24YHYB,5TFDsWVa3tDEHYu4zKjh5W0pxdCZY,[{'uid': 'a429cf63-4200-49b2-86bc-8a3ceab630de...,"{'customer_id': 'JRZK4X3HBXF6EZRACGQ24JZQ58', ...",2023-09-11_5_5TFDsWVa3tDEHYu4zKjh5W0pxdCZY_1,Match Subs 09 Sep vs Wandsworth 5s,Match Subs 09 Sep vs Wandsworth 5s,2023-09-12T10:00:00Z,SCHEDULED,Europe/London,2023-09-11T09:35:04Z,2023-09-11T09:35:30Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-09-11,False,NaN,NaN
4,inv:0-ChDwlM95rons1XoJG5fZJ9knEIMP,1,SBBJSGR24YHYB,jMfnQcteCjTj8c6ZOHIB5fczWnAZY,[{'uid': 'd6e7e7ee-8815-4d01-aad5-23becd2803b4...,"{'customer_id': 'C5R3MNHYABZ0B92S9NZYNNVGRR', ...",2023-09-11_5_jMfnQcteCjTj8c6ZOHIB5fczWnAZY_1,Match Subs 09 Sep vs Wandsworth 5s,Match Subs 09 Sep vs Wandsworth 5s,2023-09-12T10:00:00Z,SCHEDULED,Europe/London,2023-09-11T09:35:03Z,2023-09-11T09:35:30Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-09-11,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,inv:0-ChAxyXkM34z5Rr_hR1qcghr3EIMP,2,SBBJSGR24YHYB,J8M3Q8SQJERA16Tyz8YWTyueV,[{'uid': 'ec717fdb-a9fd-431a-9ee6-69e76da24af1...,"{'customer_id': '0HTAK3MQ28S69A5KBCTZ8HKHC0', ...",000008,,,NaN,PAID,Europe/London,2023-06-01T17:45:39Z,2023-06-05T06:19:45Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,https://squareup.com/pay-invoice/inv:0-ChAxyXk...,NaN
24,inv:0-ChAMb2fmrMLIoY-QsyhO607cEIMP,4,SBBJSGR24YHYB,TBKQ9UN9lcEcTA52W7S6HcpeV,[{'uid': 'dce09c3e-dc3c-4b3b-a399-313acb7090dd...,"{'customer_id': '4KEVYTZKFD01B4NE8KA0N1X8Q4', ...",000007,,,NaN,PAID,Europe/London,2023-06-01T17:45:00Z,2023-06-02T11:03:53Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,https://squareup.com/pay-invoice/inv:0-ChAMb2f...,NaN
25,inv:0-ChBSYVNkiKpyEQslJ1DOvhaHEIMP,3,SBBJSGR24YHYB,N0B8KtXp70Qg6VikEGxGHxveV,[{'uid': '58ed6fef-84c4-4a40-8323-94be5b2f0315...,"{'customer_id': 'C3S5WM4AS12RD0CZQ6DS8QBDN4', ...",000005,,,NaN,PAID,Europe/London,2023-06-01T17:26:29Z,2023-06-01T17:28:31Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,https://squareup.com/pay-invoice/inv:0-ChBSYVN...,NaN
26,inv:0-ChCP7LR57_FEOKwBMVQB0odAEIMP,2,SBBJSGR24YHYB,JMwfRq0FXCGcZqDPMquKfS6eV,[{'uid': '3891f092-3070-40e1-a0f1-6c500

# Format new subs and get customer names

In [22]:
orders_format = []
for i, order in enumerate(s_orders[:]):
    
    # print(i, '/', len(s_orders))
    new_order = {}
    
    new_order['id']=get_value(order,'id')
    new_order['created_at']=get_value(order,'updated_at')
    
    new_order['customer_id']=get_value(order,'customer_id')
    
    new_order['state']=get_value(order,'state')

    new_order['tenders']=get_value(order,'tenders')

    if not new_order['customer_id']:
        customer_ids = []
        try:
            for tender in order['tenders']:
                if get_value(tender,'customer_id'):
                    customer_ids.append(get_value(tender,'customer_id'))
            new_order['customer_id'] = customer_ids[0]
        except:
            pass

    if new_order['customer_id']:
        customer = customers_api.retrieve_customer(new_order['customer_id'])
    
        try:
            new_order['first_name'] = get_value(customer.body['customer'],'given_name')
            new_order['last_name'] = get_value(customer.body['customer'],'family_name')
        except:
            print(customer.body)
    else:
        pass
        invoice = invoices[invoices['order_id']==new_order['id']]
        
        
        if len(invoice)>0:
            cust_id = list(invoice['primary_recipient'])[0]['customer_id']
            customer = customers_api.retrieve_customer(cust_id)
            try:
                new_order['first_name'] = get_value(customer.body['customer'],'given_name')
                new_order['last_name'] = get_value(customer.body['customer'],'family_name')
            except:
                print(customer.body)
            
    for item in order['line_items']:
        line_dict = new_order.copy()
        
        line_dict['variation_name']=get_value(item,'variation_name')
        line_dict['item_name']=get_value(item,'name')
        if 'raining' in line_dict['item_name']:
            print(item)
        line_dict['note']=get_value(item,'note')
        line_dict['total_money']=item['total_money']['amount']/100
        orders_format.append(line_dict) 


orders_df = pd.DataFrame(orders_format)

{'uid': 'hKFn42MMdPLLKnzZwEwpG', 'quantity': '1', 'name': 'Training', 'base_price_money': {'amount': 500, 'currency': 'GBP'}, 'gross_sales_money': {'amount': 500, 'currency': 'GBP'}, 'total_tax_money': {'amount': 0, 'currency': 'GBP'}, 'total_discount_money': {'amount': 0, 'currency': 'GBP'}, 'total_money': {'amount': 500, 'currency': 'GBP'}, 'variation_total_price_money': {'amount': 500, 'currency': 'GBP'}, 'item_type': 'ITEM', 'total_service_charge_money': {'amount': 0, 'currency': 'GBP'}}
{'uid': 'hm4hYrL1idfdDH4j1Hgyc', 'quantity': '1', 'name': 'Training', 'base_price_money': {'amount': 500, 'currency': 'GBP'}, 'gross_sales_money': {'amount': 500, 'currency': 'GBP'}, 'total_tax_money': {'amount': 0, 'currency': 'GBP'}, 'total_discount_money': {'amount': 0, 'currency': 'GBP'}, 'total_money': {'amount': 500, 'currency': 'GBP'}, 'variation_total_price_money': {'amount': 500, 'currency': 'GBP'}, 'item_type': 'ITEM', 'total_service_charge_money': {'amount': 0, 'currency': 'GBP'}}
{'uid'

KeyboardInterrupt: 

In [23]:
list(orders_df[orders_df['item_name']=='Training']['tenders'])[0]

[{'id': 'ndZg5btuErH3BXcswsPk4nB35aHZY',
  'location_id': 'SBBJSGR24YHYB',
  'transaction_id': 'ToRlDWN1ThTb9IFQXqbF0tIEZSWZY',
  'created_at': '2023-09-10T19:10:32Z',
  'amount_money': {'amount': 500, 'currency': 'GBP'},
  'type': 'CARD',
  'card_details': {'status': 'CAPTURED',
   'card': {'card_brand': 'VISA',
    'last_4': '4154',
    'fingerprint': 'sq-1-z-eVQ6MM2t_xni0qmxPQqSf0RQlAKVDQ8E0MkJV3AUOlDGnlYcMgUg5_LLMmtsZFkg'},
   'entry_method': 'KEYED'},
  'tip_money': {'amount': 0, 'currency': 'GBP'},
  'payment_id': 'ndZg5btuErH3BXcswsPk4nB35aHZY'}]

# Get Correct columns and filter out unwanted orders

In [12]:
orders_df['customer_name']= orders_df['first_name'] + ' ' + orders_df['last_name']
match_fees_export = orders_df[(orders_df['state'].isin(['COMPLETED','OPEN'])) & (orders_df['tenders'].notnull())][['id', 'created_at', 'variation_name','item_name', 'customer_name', 'total_money', 'note']]
# match_fees_export = match_fees_export[match_fees_export['tenders'].notnull()]


match_fees_export = match_fees_export[(match_fees_export['customer_name'].notnull()) | (match_fees_export['note'].notnull())]


match_fees_export = match_fees_export[(match_fees_export['item_name'].fillna('').str.lower().str.contains('sub'))|(match_fees_export['item_name'].fillna('').str.lower().str.contains('training'))]

In [13]:
match_fees_export.columns = exist.columns

In [14]:
match_fees_export

,ID,Date,Cat,Item,Customer,Price,Notes
0,L8jU3V7OqgFJeWtNdihvOo7JPcBZY,2023-09-04T13:42:27.000Z,Full Game,Match Sub,Adam Platten,5.0,None
2,ZLfm2FvJNVRvMfdkRsl2SbE1OOPZY,2023-09-04T11:25:25.000Z,Full Game,Match Sub,Denver Lalonde,10.0,None


# Only upload new transactions

In [15]:
mg = pd.merge(left=match_fees_export, right=exist, how='left', on='ID', suffixes=['', '_old'])
not_in = mg[mg['Date_old'].isnull()][exist.columns]
len(not_in)

1

In [16]:
not_in

,ID,Date,Cat,Item,Customer,Price,Notes
0,L8jU3V7OqgFJeWtNdihvOo7JPcBZY,2023-09-04T13:42:27.000Z,Full Game,Match Sub,Adam Platten,5.0,None


In [17]:
not_in.fillna('NA', inplace=True)

# Add to Google sheet

In [18]:
append_to_sheet('Square Subs', SAMPLE_SPREADSHEET_ID_input, not_in)